Accuracy 56%

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

In [15]:
df = pd.read_csv("../Data/Combined_Delayed_2Days.csv")
df = df.replace({'Increase':1,'Decrease':-1,'No Change':0})
df = df.set_index("Date")
df.head()

,Tweets_clean_more,polarity,subjectivity,sentiment,Label_delayed_2Days
Date,,,,,
2013-01-10,to super clear i dont wish could i mandat anyt...,0.216667,0.525000,positive,-1
2019-01-10,ye get regulatori pushback may avail region go...,0.123333,0.490556,positive,1
2020-01-10,dome barrel weld made bar pretti good bar need...,0.411111,0.570988,positive,-1
2012-01-11,the exec conf room tesla use call denali i dec...,0.600000,1.000000,positive,-1
2016-01-11,sad hear david bowi die he amaz,-0.500000,1.000000,negative,1


In [16]:
x = df[['sentiment','polarity','subjectivity']]
x = pd.get_dummies(x,columns=["sentiment"])
y = df['Label_delayed_2Days']
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=123)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

In [17]:
scaler = StandardScaler()
x_scaler = scaler.fit(x_train)
x_train = x_scaler.transform(x_train)
x_test = x_scaler.transform(x_test)

In [18]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(n_estimators=20,learning_rate=learning_rate,max_features=2,max_depth=3,random_state=0)

    model.fit(x_train,y_train.ravel())
    print("Learning rate: ", learning_rate)

    print("Accuracy score (training): {0:.3f}".format(model.score(x_train, y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(model.score(x_test, y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.576
Accuracy score (validation): 0.507

Learning rate:  0.1
Accuracy score (training): 0.602
Accuracy score (validation): 0.482

Learning rate:  0.25
Accuracy score (training): 0.645
Accuracy score (validation): 0.465

Learning rate:  0.5
Accuracy score (training): 0.684
Accuracy score (validation): 0.472

Learning rate:  0.75
Accuracy score (training): 0.701
Accuracy score (validation): 0.500

Learning rate:  1
Accuracy score (training): 0.681
Accuracy score (validation): 0.496



In [20]:
x2 = df[['sentiment','polarity']]
x2 = pd.get_dummies(x2,columns=["sentiment"])
y2 = df['Label_delayed_2Days']
x2_train,x2_test,y2_train,y2_test = train_test_split(x2,y2,train_size=0.8,random_state=123)
y2_train = y2_train.values.reshape(-1,1)
y2_test = y2_test.values.reshape(-1,1)

In [22]:
scaler = StandardScaler()
x2_scaler = scaler.fit(x2_train)
x2_train = x2_scaler.transform(x2_train)
x2_test = x2_scaler.transform(x2_test)

In [23]:
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(n_estimators=20,learning_rate=learning_rate,max_features=2,max_depth=3,random_state=0)

    model.fit(x2_train,y2_train.ravel())
    print("Learning rate: ", learning_rate)

    print("Accuracy score (training): {0:.3f}".format(model.score(x2_train, y2_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(model.score(x2_test, y2_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.548
Accuracy score (validation): 0.507

Learning rate:  0.1
Accuracy score (training): 0.567
Accuracy score (validation): 0.507

Learning rate:  0.25
Accuracy score (training): 0.601
Accuracy score (validation): 0.507

Learning rate:  0.5
Accuracy score (training): 0.627
Accuracy score (validation): 0.479

Learning rate:  0.75
Accuracy score (training): 0.647
Accuracy score (validation): 0.504

Learning rate:  1
Accuracy score (training): 0.649
Accuracy score (validation): 0.514



In [24]:
x3 = df[['sentiment']]
x3 = pd.get_dummies(x3,columns=["sentiment"])
y3 = df['Label_delayed_2Days']
x3_train,x3_test,y3_train,y3_test = train_test_split(x3,y3,train_size=0.8,random_state=123)
y3_train = y3_train.values.reshape(-1,1)
y3_test = y3_test.values.reshape(-1,1)

scaler = StandardScaler()
x3_scaler = scaler.fit(x3_train)
x3_train = x3_scaler.transform(x3_train)
x3_test = x3_scaler.transform(x3_test)

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(n_estimators=20,learning_rate=learning_rate,max_features=2,max_depth=3,random_state=0)

    model.fit(x3_train,y3_train.ravel())
    print("Learning rate: ", learning_rate)

    print("Accuracy score (training): {0:.3f}".format(model.score(x3_train, y3_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(model.score(x3_test, y3_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.511
Accuracy score (validation): 0.563

Learning rate:  0.1
Accuracy score (training): 0.511
Accuracy score (validation): 0.563

Learning rate:  0.25
Accuracy score (training): 0.511
Accuracy score (validation): 0.563

Learning rate:  0.5
Accuracy score (training): 0.511
Accuracy score (validation): 0.563

Learning rate:  0.75
Accuracy score (training): 0.511
Accuracy score (validation): 0.563

Learning rate:  1
Accuracy score (training): 0.511
Accuracy score (validation): 0.563

